In [ ]:
# 1. Importing Required Libraries
import transformers  # Hugging Face transformers library for working with pre-trained models
import torch # PyTorch library for deep learning operations

# 2. Model Configuration
model_id = "aaditya/OpenBioLLM-Llama3-70B" # Specifies which model to use from HuggingFace

pipeline = transformers.pipeline(
    "text-generation",        # Task type
    model=model_id,          # Model identifier
    model_kwargs={
        "torch_dtype": torch.bfloat16  # Uses bfloat16 precision for memory efficiency
    },
    device="auto",           # Automatically selects best available device (CPU/GPU)
)

# 3. Setting up the Chat Template
messages = [
    # System prompt that defines the AI's behavior
    {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience."},
    # User's input
    {"role": "user", "content": "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"},
]

# 4. Formatting the Prompt
prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,          # Returns string instead of tokens
    add_generation_prompt=True  # Adds any necessary generation markers
)

# 5. Setting up Response Termination Conditions
terminators = [
    pipeline.tokenizer.eos_token_id,  # End of sequence token
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Custom end token
]

# 6. Generating Response
outputs = pipeline(
    prompt,
    max_new_tokens=256,      # Maximum length of generated response
    eos_token_id=terminators,  # When to stop generating
    do_sample=True,          # Enable sampling-based generation
    temperature=0.0,         # Controls randomness (0.0 = deterministic)
    top_p=0.9,              # Nucleus sampling parameter
)

# 7. Extracting and Printing Response
print(outputs[0]["generated_text"][len(prompt):])